1. Import openAI dependences (load with conda install openai)

2. Use the API Key from openAI page

In [16]:

import openai
import csv
from IPython.display import display, HTML
import ipywidgets as widgets
import json

In [13]:
#Add our API Key copied from openAI page
openai.api_key = "" 

3. Make a request

In [15]:
#Make the context of our prompt envoirement
context = """
Eres asistente en la App ServiceExpress, que conecta usuarios con profesionales de diversos trabajos, como por ejemplo, 
electricista, plomero, pintor. Tienes que evaluar los comentarios de los usuarios para decir qué tan satisfechos están 
con ese profesional y proponer puntos de mejora. Además, debes asignar un ícono que refleje la satisfacción del cliente: 
una cara feliz (satisfecho), neutra (indiferente) o triste (insatisfecho).
"""



def evaluar_comentario(profesional, cliente, comentario):
    prompt = f"{context}\n\nProfesional: {profesional}\nCliente: {cliente}\nComentario: \"{comentario}\""
    
  
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": context},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200
        )
        
        message = response.choices[0]['message']['content']
        
       
        icono = "😊" if "excelente" in comentario.lower() else "😐" if "bueno" in comentario.lower() else "😞"
        
        return message, icono
    
    except openai.error.OpenAIError as e:
        return f"Error al procesar el comentario: {e}", "❓"
    
    except Exception as e:
        return f"Error general: {e}", "❓"


def procesar_comentarios_json(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        comentarios = json.load(file)
    
    resultados = []
    
    for comentario in comentarios:
        profesional = comentario['profesional']
        cliente = comentario['cliente']
        texto_comentario = comentario['comentario']
        
        evaluacion, icono = evaluar_comentario(profesional, cliente, texto_comentario)
        resultados.append([profesional, cliente, texto_comentario, evaluacion, icono])
    
  
    with open('resultados_evaluacion.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Profesional', 'Cliente', 'Comentario', 'Evaluación', 'Ícono'])
        writer.writerows(resultados)
    
   
    for resultado in resultados:
        display(HTML(f"<b>Profesional:</b> {resultado[0]}<br><b>Cliente:</b> {resultado[1]}<br><b>Comentario:</b> {resultado[2]}<br><b>Evaluación:</b> {resultado[3]}<br><b>Ícono:</b> {resultado[4]}<br><hr>"))


json_file_path = 'comentarios.json'
procesar_comentarios_json(json_file_path)